## Optimisation of X-ray Analysis - Diagnosis of Pneumonia using machine learning

# Notebook: X-ray Image Analysis for Pneumonia Diagnosis

## Description

This notebook provides the implementation details for automating the analysis of chest X-ray images to diagnose pneumonia using Transfer Learning and Deep Template Matching. It includes the hardware requirements, necessary libraries, and steps to build and evaluate the models.

## Authors
Marzena Halama
Oliwia
Michał Gruszkowski



## Required Libraries
The following Python libraries are required for this project:
- TensorFlow
- Keras
- OpenCV
- NumPy
- Pandas
- Matplotlib
- Scikit-learn

## Models
This notebook implements and compares two machine learning models:
1. **Transfer Learning**: Utilizing pre-trained models to fine-tune on the X-ray image dataset for pneumonia detection.
2. **Deep Template Matching**: Creating templates from X-ray images and using deep learning techniques to match and identify pneumonia-related patterns.


## Dataset
The project requires downloading the chest X-ray image dataset from Kaggle. Follow these steps to obtain the dataset:

Download the dataset from [Kaggle](https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia/data).

Extract the dataset to a directory accessible by the notebook.

## Model Evaluation
To assess the effectiveness of the machine learning models, the following metrics are used:
- **Accuracy**: The proportion of correctly predicted instances out of the total instances.
- **F1 Score**: The harmonic mean of precision and recall, providing a balance between the two.

## Pre-trained Models
We will use the following pre-trained models as base models for Transfer Learning:
1. **MobileNetv2**
(the list of models used will be created here)



In [28]:
#  Importing the required libraries
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.metrics import roc_curve, auc, precision_recall_fscore_support
from tensorflow.keras.callbacks import Callback
from sklearn.preprocessing import label_binarize
from itertools import cycle
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np
from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import Callback


## Data Preprocessing
Depending on the pre-trained model used, different data preprocessing techniques may be required. Ensure to follow the specific preprocessing steps suitable for the chosen model to achieve optimal results.


In [2]:
# for MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input


### Convert grayscale images to three-channel images by stacking the grayscale data along the third dimension:

In [3]:
def convert_to_rgb(image):
    return np.stack((image,) * 3, axis=-1)

In [17]:
def preprocess_image(image):
    image = tf.image.resize(image, (224,224))
    image = tf.image.grayscale_to_rgb(image) 
    image = preprocess_input(image)
    return image

## Parameters Description:
In this section, we define the parameters that will be used in the model training and evaluation process:

- **img_size**: This defines the dimensions of the input images, which can vary depending on the model being used.

- **batch_size**: This is the number of training examples utilized in one iteration. We will use a batch size of 32.
- **n_classes**: This indicates the number of classes for classification. Since we are dealing with a binary classification problem (pneumonia vs. no pneumonia), n_classes is set to 2.
- **validation_split**: This is the proportion of the dataset to include in the validation split. We will use 20% of the data for validation.
- **seed**: This is the random seed for shuffling and transformations. Setting a seed ensures reproducibility. We use the seed value of 1337.

In [5]:
# Parameters
img_size = 224 # Dimensions of the images depending on the model
batch_size = 32  # Batch size
n_classes = 1  # Number of classes for binary classification
validation_split = 0.2  # Proportion of the dataset to use for validation
seed = 1337  # Random seed for reproducibility


In [6]:
# Base model MobileNetV2
base_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),
                                               include_top=False,  
                                               weights='imagenet') 

## the new classifier

In [35]:
x = base_model.output
x = GlobalAveragePooling2D()(x)  
x = Dense(1024, activation='relu')(x) 
x = Dropout(0.5)(x)  
predictions = Dense(n_classes, activation='sigmoid')(x)  


model = Model(inputs=base_model.input, outputs=predictions)

## Loading images from the database


In [8]:
train_ds_path = 'train'
test_ds_path = 'test'

In [9]:

def create_train_val_datasets(train_path, validation_split, seed, image_size, batch_size):
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        train_path,
        validation_split=validation_split,
        subset="training",
        seed=seed,
        image_size=image_size,
        batch_size=batch_size,
        color_mode='grayscale'
    )

    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        train_path,
        validation_split=validation_split,
        subset="validation",
        seed=seed,
        image_size=image_size,
        batch_size=batch_size,
        color_mode='grayscale'
    )

    return train_ds, val_ds

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_ds_path,
    seed=seed,
    image_size=img_size,
    batch_size=batch_size,
    color_mode='grayscale'
)


train_ds, val_ds = create_train_val_datasets(
    train_ds_path,
    validation_split,
    seed,
    image_size=img_size,
    batch_size=batch_size
)


Found 624 files belonging to 2 classes.
Found 5232 files belonging to 2 classes.
Using 4186 files for training.
Found 5232 files belonging to 2 classes.
Using 1046 files for validation.


In [20]:
normalized_ds_train = tf.data.Dataset.from_generator(
    lambda: ((preprocess_image(x), y) for x, y in train_ds),
    output_types=(tf.float32, tf.float32),
    output_shapes=((224, 224, 3), ())
)


In [24]:
def preprocess_dataset(dataset):
    images = []
    labels = []
    for batch in dataset:
        batch_images, batch_labels = batch
        for img, lbl in zip(batch_images, batch_labels):
            img = preprocess_image(img)
            images.append(img.numpy())
            labels.append(lbl.numpy())
    return np.array(images), np.array(labels)

X_train, Y_train = preprocess_dataset(train_ds)
X_val, Y_val = preprocess_dataset(val_ds)
X_test, Y_test = preprocess_dataset(test_ds)

normalized_ds_train = train_ds

In [25]:

# Extraction of X_train and Y_train from normalised data
X_train = np.concatenate([x.numpy() for x, y in normalized_ds_train], axis=0)
Y_train = np.concatenate([y.numpy() for x, y in normalized_ds_train], axis=0)

In [36]:
def convert_to_three_channels(images):
    if images.shape[-1] == 1:
        images = np.repeat(images, 3, axis=-1)
    return images

In [39]:
X_val = convert_to_three_channels(X_val)
X_train = convert_to_three_channels(X_train)

In [41]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
class F1ScoreCallback(Callback):
    def __init__(self, val_ds):
        super().__init__()
        self.val_ds = val_ds
        self.results = []

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}

        # Oblicz F1-score dla zbioru walidacyjnego
        val_predictions = (self.model.predict(self.val_ds[0]) > 0.5).astype(int)
        val_labels = self.val_ds[1]
        val_f1 = f1_score(val_labels, val_predictions)

        # Dodaj wyniki do listy
        self.results.append({
            'epoch': epoch + 1,
            'accuracy': logs.get('accuracy'),
            'val_accuracy': logs.get('val_accuracy'),
            'precision': logs.get('precision'),
            'val_precision': logs.get('val_precision'),
            'recall': logs.get('recall'),
            'val_recall': logs.get('val_recall'),
            'val_f1': val_f1
        })

        # Dodaj F1-score do logów
        logs['val_f1'] = val_f1
        print(f' — val_f1: {val_f1:.4f}')

# Inicjalizacja callbacku
f1_callback = F1ScoreCallback((X_val, Y_val))

# Trening modelu
history = model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    epochs=10,
    callbacks=[f1_callback]
)